# **Import Important Libraires**

In [19]:
import pandas as pd
import numpy as np
import missingno as msngo
import seaborn as sns
import matplotlib.pyplot as plt

# ***Collect Data About Insurance DataSet***

In [20]:
insurance=pd.read_csv('insurance.csv')
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
insurance.shape

In [ ]:
insurance.info()

In [ ]:
insurance.dtypes

In [ ]:
insurance.isnull().sum()

In [ ]:
msngo.matrix(insurance)
plt.show()

# ***visualize***

*Visualize Numerical Columns*

In [ ]:
sns.set(style="darkgrid")
sns.scatterplot(
    x='age',
    y='charges',
    data=insurance,
    hue='smoker'
)
plt.show()

In [ ]:
sns.set(style="darkgrid")
sns.scatterplot(
    x='bmi',
    y='charges',
    data=insurance,
    hue='smoker'
)
plt.show()

In [ ]:
sns.set(style="darkgrid")
sns.scatterplot(
    x='children',
    y='charges',
    data=insurance,
    hue='smoker'
)
plt.show()

In [ ]:
sns.set(style="darkgrid")
plt.figure(figsize=(10, 6))


sns.regplot(x="age", y="charges", data=insurance, label="Age")
sns.regplot(x="bmi", y="charges", data=insurance, label="BMI")

plt.title("Age and BMI vs Charges")
plt.xlabel("Age and BMI")
plt.ylabel("Charges")

plt.legend()

plt.show()

In [ ]:
sns.set(style="darkgrid")
sns.pairplot(
    data=insurance,
    hue='smoker'
)
plt.show()

*Visualize Categorical Columns*

In [ ]:
sns.set(style="darkgrid")
sns.violinplot(data=insurance, x="smoker", y="charges", inner="point")
plt.show()

In [ ]:
sns.set(style="darkgrid")
sns.violinplot(data=insurance, x="sex", y="charges", inner="point")
plt.show()

In [ ]:
sns.set(style="darkgrid")
sns.countplot(
    x='region',
    data=insurance,
    hue='smoker'
)
plt.show()

In [ ]:
sns.set(style="darkgrid")
sns.countplot(
    x='sex',
    data=insurance,
    hue='smoker'
)
plt.show()

# ***Prepare the Data***

In [35]:
x = insurance.drop(['charges'], axis=1)
y = insurance['charges']

In [36]:
categorical_features = ['sex', 'smoker', 'region']
numerical_features = ['age', 'bmi']

In [37]:
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(
    x,
    y,
    test_size=.2,
    random_state=42
)

In [38]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline


categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [39]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# ***Model***

In [40]:
from sklearn.ensemble import GradientBoostingRegressor

model=GradientBoostingRegressor()

param_grid = {
    'GradientBoostingRegressor__n_estimators': [80, 100, 120],
    'GradientBoostingRegressor__learning_rate': [0.01, 0.1, 0.2],
    'GradientBoostingRegressor__max_depth': [3, 4, 5]
}

In [41]:
pipeline=Pipeline([
        ('preprocessor', preprocessor),
        ('GradientBoostingRegressor', model)
    ])

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV


grid_search = GridSearchCV(pipeline,param_grid,cv=5, scoring=make_scorer(mean_squared_error, greater_is_better=False))

grid_search.fit(x, y)

In [ ]:
best_estimator=grid_search.best_estimator_
best_score= -grid_search.best_score_
print(best_estimator)
print('best score is : ' + str(best_score))

In [44]:
y_pred=grid_search.predict(x_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f'Test Mean Squared Error: {mse}')

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)

print(f"R-squared: {r2:.2f}")